# Desafío 01

## Integrantes

- Acevedo Zain, Gaspar (acevedo.zain.gaspar@gmail.com)

## Consignas

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


## Imports y carga de datos

In [1]:
%pip install numpy scikit-learn

In [79]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

from sklearn.datasets import fetch_20newsgroups
import numpy as np

from sklearn.utils import Bunch
from scipy.sparse import csr_matrix
from typing import Tuple

import random

In [4]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Resolución ejercicio 1

***Enunciado***

- Vectorizar documentos.
- Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
- Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

Comenzamos inicializando un vectorizador del tipo [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) con los parámetros por defecto.

In [11]:
tfidVectorizer_01 = TfidfVectorizer()

Una vez inicializado el vectorizador, se procede a vectorizar el set de entrenamiento (`newsgroups_train`).

In [12]:
X_train = tfidVectorizer_01.fit_transform(newsgroups_train.data)

Se muestra que hay $11314$ documentos vectorizados en el set de entrenamiento.

In [16]:
cant_docs = X_train.shape[0]
print(f"Cantidad de documentos en train: {cant_docs}")

Cantidad de documentos en train: 11314


Se muestra que el tamaño del vocabulario en el set de entrenamiento es de $101631$.

In [137]:
tam_vocab = X_train.shape[1]
print(f"Tamaño del vocabulario en train: {tam_vocab}")

Tamaño del vocabulario en train: 101631


Se crea a continuación un diccionario que va de índices a términos (código dado por la materia).

In [138]:
idx2word = {v: k for k,v in tfidVectorizer_01.vocabulary_.items()}

A continuación, se seleccionan al azar $5$ documentos vectorizados del set de entrenamiento.

In [17]:
random_idxs = [random.randint(0, cant_docs -1) for _ in range(5)]

In [48]:
corpus = newsgroups_train

### Funciones de utilidad

Se definen las siguientes funciones de utilidad:

- `get_documents_similarity`, la cual, dados una `matriz documento-término` y un índice de un documento de la misma, obtiene la `similitud coseno` entre dicho documento y el resto de los documentos de la matriz, como así también los índices de estos documentos.
- `get_k_most_similar_documents`, la cual utiliza la función anterior, y devuelve a los $K$ documentos con mayor `similitud coseno` junto a sus índices. Por defecto $K = 5$.
- `imprimir_informe`, la cual devuelve un informe de un documento de un corpus, al cual se compara utilizando la similitud coseno con otros documentos del corpus.
- `get_common_words`, la cual devuelve los términos que tienen en común dos documentos vectorizados.

In [72]:
def get_documents_similarity(matriz_dt: csr_matrix, idx: int) -> Tuple[np.ndarray, np.ndarray]:
  """
  Devuelve la similitud coseno de un documento respecto al resto de los
    documentos de una matriz documento-término.

  Args:
    matriz_dt (csr_matrix): Matrix documento-término a partir de la cual se
      obtendrá la similitud coseno de un documento, dado su índice.
    idx (int): índice del documento sobre el cual se trabajará.
  Returns:
    Tuple[np.ndarray, np.ndarray]:
      Elem 0: documentos ordenados de mayor a menor similitud coseno.
      Elem 1: argumentos correspondientes a Elem 0.
  Raises:
    IndexError, si el parámetro `idx` está fuera del rando de índices de
      `matriz_dt`.
  """

  if idx >= matriz_dt.shape[0]:
    raise IndexError(f"El parámetro `idx` está fuera del rango del corpus \
    ({matriz_dt.shape[0]}).")

  documents_similarity = cosine_similarity(matriz_dt[idx], matriz_dt)[0]

  sorted_docs = np.sort(documents_similarity)[::-1]
  arg_docs = np.argsort(documents_similarity)[::-1]
  return sorted_docs, arg_docs


In [73]:
def get_k_most_similar_documents(matriz_dt: csr_matrix, idx: int, K: int = 5) -> Tuple[np.ndarray, np.ndarray]:
  """
  Dado una matríz documento-término y un índice correspondiente a uno de sus documentos, devuelve
  los K documentos con mayor similitud coseno respecto del documento indicado.

  Args:
    matriz_dt (csr_matrix): Matrix documento-término a partir de la cual se
      obtendrá la similitud coseno de un documento, dado su índice.
    idx (int): índice del documento sobre el cual se trabajará.
    K (int): número de documentos con mayor similitud que se quiere encontrar.
      Por defecto es 5.
  Returns:
    np.ndarray: array de longitud K que contiene la similitud coseno del
      documento a analizar respecto al resto, ordenados de mayor a menor.
    Tuple[np.ndarray, np.ndarray]:
      Elem 0: `K` documentos ordenados de mayor a menor similitud coseno.
      Elem 1: argumentos correspondientes a Elem 0.
  Raises:
    IndexError, si el parámetro `idx` está fuera del rando de índices de
      `matriz_dt`.
  """

  if idx >= matriz_dt.shape[0]:
    raise IndexError(f"El parámetro `idx` está fuera del rango de la matriz \
    documento-término ({matriz_dt.shape[0]}).")

  docs_similarity, docs_args  = get_documents_similarity(matriz_dt=matriz_dt, idx=idx)

  # El índice 0 corresponde al propio documento
  return docs_similarity[1:K + 1], docs_args[1:K + 1]

In [111]:
def imprimir_informe(corpus: Bunch, idx: int, similar_docs: np.ndarray, similar_docs_idxs: np.ndarray) -> str:
  """
  Esta función permite mostrar un informe correspondiente al documento de un
  corpus dado, en donde se muestran los K documentos con mayor similitud coseno.

  Args:
    corpus (Bunch): corpus sobre el cual se trabaja.
    idx (int): índice del documento sobre el cual se hace el análisis.
    similar_docs (np.ndarray): array que contiene los K documentos con mayor
      similitud coseno respecto del documento de análisis.
    similar_docs_idxs (np.ndarray): array que contiene los índices correspondientes
      a los documentos de `similar_docs`.

  Returns:
    string: Informe, con tabla comparativa de documentos.
  """
  doc_data = corpus.data[idx]
  doc_target_idx = corpus.target[idx]
  doc_type = corpus.target_names[doc_target_idx]

  informe = f"En esta sección se realiza el análisis del documento con índice ${idx}$\n\n"
  informe += f"Este es un documento del tipo `{doc_type}`.\n\n"
  informe += f"A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.\n\n"

  tabla_comparativa = f"| Índice documento | Tipo de documento | Similitud coseno con documento ${idx}$ | Coinciden los tipos de documento? |\n"
  tabla_comparativa += f"| --- | --- | --- | --- |\n"

  for i in range(len(similar_docs)):
    # Similitud coseno entre documentos
    similarity = similar_docs[i]
    # Índice dentro del corpus del documento con el cual se comparó
    similarity_idx = similar_docs_idxs[i]
    # Índice del tipo de documento contra el cual se comparó
    similarity_target_idx = corpus.target[similarity_idx]
    # Tipo de documento contra el cual se comparó
    similarity_type = corpus.target_names[similarity_target_idx]

    same_type = "Si" if doc_type == similarity_type else "No"

    tabla_comparativa += f"| ${similarity_idx}$ | {similarity_type} | ${np.round(similarity, 4)}$ | {same_type} |\n"

  return informe + tabla_comparativa

In [142]:
def get_common_words(vectorized_corpus: csr_matrix, idx1: int, idx2: int, idx2word: dict) -> list:
  """
  Devuelve los términos comunes de dos documentos vectorizados.

  Args:
    vectorized_corpus (csr_matrix): Corpus (vectorizado) al que corresponden
      los documentos.
    idx1 (int): índice dentro del vectorized corpus del primer documento.
    idx2 (int): índice dentro del vectorized corpus del segundo documento.
    idx2word: diccionario que va de índices a términos del corpus.

  Returns:
    list: lista con términos en común.
  """
  # Términos del documento 1 (o documento base)
  _, cols1 = (vectorized_corpus[idx1]).nonzero()
  # Términos del documento 2
  _, cols2 = (vectorized_corpus[idx2]).nonzero()

  # Los trato como `sets` para facilitar la `intersección`
  cols1set = set(cols1)
  cols2set = set(cols2)
  common_cols = list(cols1set.intersection(cols2set))

  common_words = []

  for col in common_cols:
    common_words.append(idx2word[col])

  return common_words

### Análisis del documento con índice $11097$

En esta sección se realiza el análisis del documento con índice $11097$

Este es un documento del tipo `soc.religion.christian`.

A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.

| Índice documento | Tipo de documento | Similitud coseno con documento $11097$ | Coinciden los tipos de documento? |
| --- | --- | --- | --- |
| $4626$ | soc.religion.christian | $0.2651$ | Si |
| $11117$ | soc.religion.christian | $0.2036$ | Si |
| $7358$ | talk.religion.misc | $0.2027$ | No |
| $4800$ | soc.religion.christian | $0.2022$ | Si |
| $8807$ | talk.religion.misc | $0.1759$ | No |

In [112]:
idx = random_idxs[0]
similar_docs, similar_docs_idx = get_k_most_similar_documents(matriz_dt=X_train, idx=idx)
informe = imprimir_informe(corpus=corpus, idx=idx, similar_docs=similar_docs, similar_docs_idxs=similar_docs_idx)

In [113]:
print(informe)

En esta sección se realiza el análisis del documento con índice $11097$

Este es un documento del tipo `soc.religion.christian`.

A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.

| Índice documento | Tipo de documento | Similitud coseno con documento $11097$ | Coinciden los tipos de documento? |
| --- | --- | --- | --- |
| $4626$ | soc.religion.christian | $0.2651$ | Si |
| $11117$ | soc.religion.christian | $0.2036$ | Si |
| $7358$ | talk.religion.misc | $0.2027$ | No |
| $4800$ | soc.religion.christian | $0.2022$ | Si |
| $8807$ | talk.religion.misc | $0.1759$ | No |



Como se puede observar, para el primer documento analizado, el cual tiene el índice $11097$, de los $5$ documentos con mayor similitud coseno se encontró que $3$ de ellos tienen el mismo tipo (`soc.religion.christian`), mientras que el resto no, aunque son de una *temática* similar (`talk.religion.misc`).

A continuación se muestra el contenido de todos los documentos utilizados en esta comparación.

#### Comparaciones entre documentos

Contenido del documento analizado/documento base (índice $11097$)

- Tipo de documento: `soc.religion.christian`.

Como se puede observar, aparecen *términos* como **God**, **Heaven**, **mercy**, lo cual dan a entender que es un documento que referencia a la fe Cristiana (de ahí su tipo).

In [115]:
print(corpus.data[idx])


I have also heard it called an expression of mercy, because Heaven would be
far more agonizing for those who had rejected God.



Contenido del documento con índice $4626$:
- Similitud coseno: $0.2651$.
- Tipo de documento: `soc.religion.christian`.
- Tiene el mismo tipo que el documento base?: ***Si***.

En este documento aparecen los términos **God**, **mercy** y **Hell**, el cual da a enteder que es un documento que referencia a la fe Cristiana (tipo `soc.religion.christian`).

Los términos en común con el documento base son: `god`, `be`, `an`, `expression`, `mercy`, `of`, `because`.

Varios de estos términos podrían aportar poco valor (`be`, `an`), ya que suelen repetirse en documentos de distinto tipo dentro del corpus, pero los que sí podrían aportar bastante valor son `god` y `mercy`.

In [116]:
print(corpus.data[similar_docs_idx[0]])

Quoth the Moderator:


In a short poem ("God in His mercy made / the fixed pains of Hell"),
C. S. Lewis expresses an idea that I'm sure was current among others,
but I haven't be able to find its source:

that even Hell is an expression of mercy, because God limits the amount
of separation from Him, and hence the amount of agony, that one can
achieve.



In [147]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[0], idx2word=idx2word)
print(common_words)


['god', 'be', 'an', 'expression', 'mercy', 'of', 'because']


Contenido del documento con índice $11117$:
- Similitud coseno: $0.2036$.
- Tipo de documento: `soc.religion.christian`.
- Tiene el mismo tipo que el documento base?: ***Si***.

En este documento aparecen los términos **God**, **Heaven**, **faith**, **christians**, entre otros, los cuales dan a enteder que es un documento que referencia a la fe Cristiana (tipo `soc.religion.christian`).

Los términos en común con el documento base son: `god`, `be`, `heaven`, `who`, `would`, `of`, `because`, `have`, `it`.

In [148]:
print(corpus.data[similar_docs_idx[1]])

Here's how I talk to non-Christians who are complaining about Hell.

ME:	"Do you believe you're going to Heaven?"
HIM:	"I don't believe in Heaven."
ME:	"So are you going there?"
HIM:	"If there was a heaven, I would."
ME:	"But since there isn't a Heaven, you're not going there, are you?"
HIM:	"No."

  The point is that Heaven is based on faith--if you don't believe in heaven,
there's no way you're going to be in it.
  Of course, the next step is, "I don't believe in Hell either, so why will I
be there?"  It seems to me that Hell is eternal death and seperation from God.
Most atheists do believe that when they die they will die forever, and never
see God--so they do, in fact, believe that they're going to Hell.
  Hell doesn't have to be worse than earth to be Hell--because it's eternal, 
and it's a lot worse than Heaven.  That's the only comparison that matters.


In [149]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[1], idx2word=idx2word)
print(common_words)


['god', 'be', 'heaven', 'who', 'would', 'of', 'because', 'have', 'it']


Contenido del documento con índice $7358$:
- Similitud coseno: $0.2027$.
- Tipo de documento: `talk.religion.misc`.
- Tiene el mismo tipo que el documento base?: ***No***.

En este documento aparecen los términos **God**, **Jesus**,entre otros, los cuales dan a enteder que es un documento que referencia a la religión (tipo `talk.religion.misc`).

Los términos en común con el documento base son: `god`, `be`, `more`, `for`, `an`, `also`, `expression`, `those`, `heaven`, `who`, `of`, `because`, `have`, `it`.

Si bien hay muchos términos en común, la mayoría son conectores (como `of`, `for`, `an`) o términos que aportan poco valor (como `also`, `those`, `it`).

In [153]:
print(corpus.data[similar_docs_idx[2]])

iank@microsoft.com (Ian Kennedy) writes...


More along the lines of Hebrews 12:25-29, I reckon...

	See that you refuse not him that speaks. For if they
	escaped not who refused him that spake on earth, much 
	more shall not we escape, if we turn away from him that 
	speaks from heaven:

	Whose voice then shook the earth: but now he has promised,
	saying, Yet once more I shake not the earth only, but also
	heaven.

	And this word, Yet once more, signifies the removing of
	those things that are shaken, as of things that are made,
	that those things which cannot be shaken may remain.

	Wherefore we receiving a kingdom which cannot be moved, 
	let us have grace, whereby we may serve God acceptably 
	with reverence and godly fear:

	For our God is a consuming fire.


Or 2nd Thessalonians 1:7-10...

	And to you who are troubled rest with us, when the Lord
	Jesus shall be revealed from heaven with his mighty angels,
 	In flaming fire taking vengeance on them that know not God,
	and that obe

In [151]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[2], idx2word=idx2word)
print(common_words)

['god', 'be', 'more', 'for', 'an', 'also', 'expression', 'those', 'heaven', 'who', 'of', 'because', 'have', 'it']


Contenido del documento con índice $4800$:
- Similitud coseno: $0.2022$.
- Tipo de documento: `soc.religion.christian`.
- Tiene el mismo tipo que el documento base?: ***Si***.

En este documento aparecen los términos **God**, **Christians**, **Hell** ,entre otros, los cuales dan a enteder que es un documento que referencia a la religión cristiana (tipo `soc.religion.christian`).

Los términos en común con el documento base son: `god`, `be`, `for`, `an`, `heaven`, `called`, `who`, `of`, `because`, `it`.

La mayoría son términos que aportan poco valor (como `for`, `an`, entre otros). Los que sí podrían aportar bastante valor son `god` y `heaven`.

In [157]:
print(corpus.data[similar_docs_idx[3]])


: >People who reject God don't want to be wth Him in heaven.  We spend our 
: >lives choosing to be either for Him or against Him.  God does not force 
: >Himself on us.

: I must say that I am shocked. My impression has been that Jayne Kulikaskas
: usually writes this much less offensive and ludicrous than this. I am not
: saying that the offensiveness is intentional, but it is clear and it is
: something for Christians to consider.

Jayne stands in pretty good company.  C.S. Lewis wrote a whole book
promoting the idea contained in her first sentence quoted above.  It is
called "The Final Divorce".  Excellent book on the subject of Heaven and
Hell, highly recommended.  It's an allegory of souls who are invited, indeed
beseeched to enter Heaven, but reject the offer because being with God in
Heaven means giving up their false pride.



In [156]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[3], idx2word=idx2word)
print(common_words)

['god', 'be', 'for', 'an', 'heaven', 'called', 'who', 'of', 'because', 'it']


Contenido del documento con índice $8807$:
- Similitud coseno: $0.1759$.
- Tipo de documento: `talk.religion.misc`.
- Tiene el mismo tipo que el documento base?: ***No***.

En este documento aparecen los términos **God**, **faith** ,entre otros, los cuales dan a enteder que es un documento que referencia a la religión en general (tipo `talk.religion.misc`).

Los términos en común con el documento base son: `god`, `be`, `for`, `an`, `heaven`, `called`, `who`, `of`, `because`, `it`.

La mayoría de estos términos aportan poco valor, ya que se repiten en la mayoría de los documentos del corpus (`of`, `it`, `because`).

El único término que sí podría aportar valor es `god`, de ahí a que este documento sea el que menor similitud coseno (de los $5$ elegidos) tenga con el documento base.

In [160]:
print(corpus.data[similar_docs_idx[4]])


And does it not say in scripture that no man knows the hour of His coming, not
even the angels in Heaven but only the Father Himself?  DK was trying to play
God by breaking the seals himself.  DK killed himself and as many of his
followers as he could.  BTW, God did save the children.  They are in Heaven,
a far better place.  How do I know?  By faith.

God be with you,


In [159]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[3], idx2word=idx2word)
print(common_words)

['god', 'be', 'for', 'an', 'heaven', 'called', 'who', 'of', 'because', 'it']


### Análisis del documento con índice $3377$

En esta sección se realiza el análisis del documento con índice $3377$

Este es un documento del tipo `rec.sport.hockey`.

A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.

| Índice documento | Tipo de documento | Similitud coseno con documento $3377$ | Coinciden los tipos de documento? |
| --- | --- | --- | --- |
| $1573$ | rec.sport.hockey | $0.6856$ | Si |
| $7070$ | rec.sport.hockey | $0.68$ | Si |
| $6985$ | rec.sport.hockey | $0.6298$ | Si |
| $3576$ | rec.sport.hockey | $0.5444$ | Si |
| $1197$ | rec.sport.hockey | $0.3814$ | Si |

In [161]:
idx = random_idxs[1]
similar_docs, similar_docs_idx = get_k_most_similar_documents(matriz_dt=X_train, idx=idx)
informe = imprimir_informe(corpus=corpus, idx=idx, similar_docs=similar_docs, similar_docs_idxs=similar_docs_idx)

In [162]:
print(informe)

En esta sección se realiza el análisis del documento con índice $3377$

Este es un documento del tipo `rec.sport.hockey`.

A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.

| Índice documento | Tipo de documento | Similitud coseno con documento $3377$ | Coinciden los tipos de documento? |
| --- | --- | --- | --- |
| $1573$ | rec.sport.hockey | $0.6856$ | Si |
| $7070$ | rec.sport.hockey | $0.68$ | Si |
| $6985$ | rec.sport.hockey | $0.6298$ | Si |
| $3576$ | rec.sport.hockey | $0.5444$ | Si |
| $1197$ | rec.sport.hockey | $0.3814$ | Si |



Como se puede observar, todos los documentos tienen el mismo tipo `rec.sport.hockey`.

Además, el documento con mayor similitud coseno es el $1573$, con $0.6856$, mientras que el menor es el $1197$, con $0.3871$.

Igualmente, los valores son relativamente altos, en comparación a los obtenidos en el caso de análisis anteriors (índice $11097$).

#### Comparaciones entre documentos

Contenido del documento analizado/documento base (índice $3377$)

- Tipo de documento: `rec.sport.hockey`.

In [163]:
print(corpus.data[idx])

Apparently, Part 2 (defensemen numbered 2 through 19) was lost when I posted
it; to make things worse, I lost my own copy.  I have asked on the Sharks
mailing list, on which it did get out, to see if someone can mail me a copy
back.  If someone responds, I will repost it when I get it; otherwise, I will
re-write it in a day or two and post it.  Sorry about that.

#8	LARRY DEPALMA		Season:	3rd
Acquired:	'91-92, free agent from Minnesota
Grade:		I (C-/D+)

No netter/fan watching the Sharks, including me, knows where DePalma was at
the end of the season; the Sharks appeared to still have his rights, but he
wasn't in San Jose, he wasn't in Kansas City, and his name hasn't appeared on
the waiver wire.  And that kind of invisibility was prevalent for the most part
for DePalma.  After being called up October 30 against the Tampa Bay Lightning,
DePalma had an excellent week, scoring 5 points in 3 games while playing good
defense.  Unfortunately, the 27-year-old DePalma then disappeared the res

Contenido del documento con índice $1573$:
- Similitud coseno: $0.6856$.
- Tipo de documento: `rec.sport.hockey`.
- Tiene el mismo tipo que el documento base?: ***Si***.

A continuación se muestran los términos comúnes con el documento base

In [165]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[0], idx2word=idx2word)
print(common_words)

['healthy', 'time', '1st', 'all', 'decimated', 'turn', 'san', 'times', 'now', 'first', 'only', 'demonstrated', 'size', '38', 'would', '2nd', 'because', 'high', 'fan', 'there', 'year', 'future', 'probably', 'excellent', '45', 'of', 'and', 'quite', 'entry', 'down', 'game', 'was', 'lbs', 'offense', 'scoring', 'offensive', 'games', 'been', 'if', '29', 'successful', 'it', 'grade', 'him', 'lost', 'more', 'early', 'played', 'judge', 'me', 'playing', 'sharks', 'accused', 'playmaking', 'with', '91', 'appeared', 'at', 'appears', 'skills', 'pick', 'for', 'when', 'may', 'around', 'many', 'write', 'this', 'no', 'next', 'his', 'which', 'draft', 'although', 'way', 'both', '20', '92', 'so', '15', 'either', 'trade', 'impress', 'but', 'impressive', 'thoroughly', 'defense', 'has', 'those', 'defensive', 'defensively', 'out', 'from', 'hasn', '93', 'just', 'had', 'acquired', 'did', 'to', 'who', 'expansion', 'strong', 'cup', '25', 'in', 'kansas', 'as', 'nhl', 'or', 'needs', 'have', 'fighting', 'pederson', 'e

Contenido del documento con índice $7070$:
- Similitud coseno: $0.68$.
- Tipo de documento: `rec.sport.hockey`.
- Tiene el mismo tipo que el documento base?: ***Si***.

A continuación se muestran los términos comúnes con el documento base

In [166]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[1], idx2word=idx2word)
print(common_words)

['healthy', 'time', 'one', '1st', 'then', 'all', 'times', 'first', 'only', 'total', 'tremendously', 'would', 'rest', '2nd', 'because', '17', '23', 'there', 'rob', 'year', 'probably', 'excellent', 'of', 'and', 'entry', 'down', 'free', 'hard', 'was', 'game', 'scorer', 'offense', 'games', 'offensive', 'physical', 'if', 'offensively', 'star', 'wasn', 'him', 'grade', 'it', 'more', 'play', 'miss', 'before', 'ready', 'playing', 'sharks', 'playmaking', 'with', '91', 'at', 'two', 'appears', 'skills', 'pick', 'for', 'lack', 'also', 'when', 'sending', 'leading', 'this', 'no', 'next', 'his', 'which', 'draft', 'although', 'being', 'while', 'both', 'so', 'david', '92', '15', 'but', 'defense', 'defensemen', 'defensive', 'defensively', 'out', 'most', 'least', 'from', 'improved', '84', 'position', '4th', 'just', 'had', 'acquired', 'training', 'did', 'to', 'who', 'didn', 'expansion', 'mvp', '19', 'back', '25', 'mark', 'in', 'kansas', 'as', 'goal', 'goals', 'or', 'needs', 'great', 'end', 'serious', 'part

Contenido del documento con índice $6985$:
- Similitud coseno: $0.6298$.
- Tipo de documento: `rec.sport.hockey`.
- Tiene el mismo tipo que el documento base?: ***Si***.

A continuación se muestran los términos comúnes con el documento base

In [167]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[2], idx2word=idx2word)
print(common_words)

['healthy', 'time', 'one', '1st', 'then', 'all', 'times', '28', 'now', 'first', 'demonstrated', 'would', 'perhaps', 'because', 'blades', 'year', 'spot', 'of', 'and', 'own', 'free', 'hard', 'game', 'was', 'games', 'been', 'if', 'star', 'wasn', 'tampa', 'it', 'grade', 'him', 'yet', 'more', 'play', 'miss', 'early', 'apparently', 'me', 'instead', 'playing', 'sharks', '3rd', 'with', 'playoff', '91', 'at', 'two', 'for', 'also', 'clearly', 'may', 'key', 'this', 'next', 'his', 'which', 'draft', 'being', 'get', '92', 'but', 'defense', 'has', 'out', 'least', 'getting', 'most', 'from', 'just', 'had', 'acquired', 'did', 'to', 'who', 'mvp', 'expansion', 'strong', 'cup', 'expected', 'in', 'kansas', 'do', 'puck', 'as', 'nhl', 'through', 'goaltenders', 'throughout', 'or', 'close', 'needs', 'have', 'another', 'end', 'how', 'bay', 'after', 'you', 'dispersal', '13', 'thus', 'against', 'pretty', 'season', 'ice', 'major', '30', 'team', 'make', 'city', 'agent', 'will', 'last', 'my', 'making', 'than', 'good'

Contenido del documento con índice $3576$:
- Similitud coseno: $0.5444$.
- Tipo de documento: `rec.sport.hockey`.
- Tiene el mismo tipo que el documento base?: ***Si***.

A continuación se muestran los términos comúnes con el documento base

In [168]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[3], idx2word=idx2word)
print(common_words)

['one', 'then', 'sorry', 'times', 'now', 'size', 'only', 'exactly', 'looked', 'rest', 'because', 'there', 'disappeared', 'future', 'kisio', 'probably', 'of', 'and', 're', 'quite', 'coffey', 'down', 'was', 'far', 'games', 'been', 'if', 'wasn', 'him', 'it', 'lost', 'himself', 'more', 'play', 'played', 'judge', 'ready', 'me', 'playing', 'sharks', 'with', 'suspended', 'at', 'rep', 'for', 'when', 'around', 'clearly', 'this', 'no', 'his', 'which', 'although', 'so', 'based', 'but', 'sent', 'has', 'out', 'least', 'always', 'getting', 'from', 'just', 'had', 'did', 'to', 'didn', 'expansion', 'back', 'in', 'kansas', 'puck', 'as', 'goal', 'through', 'nhl', 'have', 'another', 'carter', 'actually', 'how', 'club', 'however', 'by', 'you', 'dody', '13', 'about', '16', 'larry', 'fact', 'against', 'season', 'ice', 'unfortunately', 'too', 'team', 'make', 'city', 'some', 'my', 'than', 'injured', 'good', 'injuries', 'very', 'injury', 'is', 'still', 'that', 'he', 'don', 'not', 'the', 'point', 'well', 'best',

Contenido del documento con índice $1197$:
- Similitud coseno: $0.3814$.
- Tipo de documento: `rec.sport.hockey`.
- Tiene el mismo tipo que el documento base?: ***Si***.

A continuación se muestran los términos comúnes con el documento base

In [169]:
common_words = get_common_words(vectorized_corpus=X_train, idx1=idx, idx2=similar_docs_idx[4], idx2word=idx2word)
print(common_words)

['time', 'one', '1st', 'then', 'all', 'now', 'first', 'only', '14', 'total', '38', 'look', 'would', 'touch', 'high', 'there', '23', 'year', 'future', 'of', 'and', 're', 'entry', 'own', 'hard', 'score', 'game', 'was', 'list', 'exchange', 'games', 'scoring', 'offensive', 'been', 'if', 'wasn', 'successful', 'it', 'him', 'more', 'play', 'before', 'me', 'playing', 'with', '91', 'at', 'things', 'pick', 'for', 'within', 'also', '18', '55', 'many', 'third', 'sending', 'leader', 'this', 'no', 'next', 'dave', 'his', 'which', 'draft', 'being', 'way', 'get', '20', '92', 'hartford', '15', 'either', 'but', 'defense', 'defensemen', 'has', 'defensive', 'out', 'least', 'getting', 'most', 'from', 'mostly', '84', '4th', '93', 'just', 'had', 'did', 'to', 'led', 'who', 'didn', '19', '40', 'strong', 'mark', 'in', 'puck', 'goal', 'as', 'nhl', 'goals', 'have', 'we', 'another', 'end', 'mail', 'mailing', 'until', 'how', 'club', '58', 'part', 'by', 'after', 'you', '41', '11', '13', 'about', 'much', 'power', 'aga

### Análisis del documento con índice $8880$

En esta sección se realiza el análisis del documento con índice $8880$

Este es un documento del tipo `misc.forsale`.

A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.

| Índice documento | Tipo de documento | Similitud coseno con documento $8880$ | Coinciden los tipos de documento? |
| --- | --- | --- | --- |
| $11313$ | rec.motorcycles | $0.0$ | No |
| $16$ | comp.graphics | $0.0$ | No |
| $17$ | rec.autos | $0.0$ | No |
| $18$ | sci.electronics | $0.0$ | No |
| $19$ | comp.windows.x | $0.0$ | No |

Como se puede observar, este documento se caracteriza por ***NO*** tener documentos similares.

Al analizar el contenido del mismo, se observa que éste consta de los caracteres de `\n`, el cual consiste en un salto de línea.

Es decir, estamos ante un caso erróneo que debió haberse eliminado en un preprocesamiento de datos.

Por este motivo, NO se continúa con el análisis de este documento.

In [172]:
idx = random_idxs[2]
similar_docs, similar_docs_idx = get_k_most_similar_documents(matriz_dt=X_train, idx=idx)
informe = imprimir_informe(corpus=corpus, idx=idx, similar_docs=similar_docs, similar_docs_idxs=similar_docs_idx)

In [173]:
print(informe)

En esta sección se realiza el análisis del documento con índice $8880$

Este es un documento del tipo `misc.forsale`.

A continuación, se muestran los  5  documentos con mayor similitud coseno, dentro del corpus de ***entrenamiento***.

| Índice documento | Tipo de documento | Similitud coseno con documento $8880$ | Coinciden los tipos de documento? |
| --- | --- | --- | --- |
| $11313$ | rec.motorcycles | $0.0$ | No |
| $16$ | comp.graphics | $0.0$ | No |
| $17$ | rec.autos | $0.0$ | No |
| $18$ | sci.electronics | $0.0$ | No |
| $19$ | comp.windows.x | $0.0$ | No |



In [180]:
corpus.data[idx]

'\n'

### Análisis del documento con índice $4$

### Análisis del documento con índice $5$

## Resolución ejercicio 2

## Resolución ejercicio 3

## Resolución ejercicio 4